Weighted proportion of individuals by county.

Model number of mosquitos

1 in 5 have a fever.

1 in 150 are hospitalized.

Probability someone dies.

Population

Cost per spray (sq_mile)

cost per zip code


$21,000 for medical care

$5,200/sq. mile spray cost

In [1]:
import pandas as pd
pd.options.display.max_columns = None

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
! ls ../data

WNV_Pop_Age.csv              prediction_probabilities.csv
WNV_Pop_Age_Temp.csv         processed_test.csv
__pycache__                  processed_train.csv
census_age.csv               sampleSubmission.csv
census_info.csv              spray.csv
data-clean.ipynb             spray_traps.csv
demographics.csv             test.csv
distance-mapping.ipynb       train.csv
kaggle_submission.csv        weather.csv
model_data.csv               zip_codes.csv


In [3]:
info = pd.read_csv('../data/census_info.csv').drop(['Unnamed: 0', 'GEO.id', 'GEO.display-label'], axis=1)
age = pd.read_csv('../data/census_age.csv').drop(['Unnamed: 0', 'GEO.id', 'GEO.display-label'], axis=1)

In [4]:
df = info.merge(age, on='zip_code', how='left')

In [5]:
import numpy as np
df = df.replace(['-', 'N'], np.NaN)
df = df.apply(np.float32)
df['zip_code'] = df['zip_code'].apply(np.int32)

In [6]:
df.dtypes

zip_code                     int64
poulation                  float32
percent_employed           float32
inc_<10k                   float32
inc_10-15k                 float32
inc_15-25k                 float32
median_inc                 float32
mean_inc                   float32
percent_no_health          float32
percent_public_health      float32
total_population           float32
percent_age_under_5        float32
percent_age_5-9            float32
percent_age_10-14          float32
percent_age_15-19          float32
percent_age_20-24          float32
percent_age_25-34          float32
percent_age_35-44          float32
percent_age_45-54          float32
percent_age_55-59          float32
percent_age_60-64          float32
percent_age_65-74          float32
percent_age_75-84          float32
percent_age85_and_over     float32
percent_age_65_and_over    float32
median_age                 float32
dtype: object

In [7]:
df['population_porportion'] = df['total_population'] / df['total_population'].values.sum()
df['percent_age_over_65'] = df['percent_age_65-74'] + df['percent_age_75-84'] + df['percent_age85_and_over']

df['percent_age_under_14'] = df['percent_age_under_5'] + df['percent_age_5-9'] + df['percent_age_10-14']


df['percent_low_income'] = df['inc_<10k'] + df['inc_10-15k']


In [8]:
df.head(1)


,zip_code,poulation,percent_employed,inc_<10k,inc_10-15k,inc_15-25k,median_inc,mean_inc,percent_no_health,percent_public_health,total_population,percent_age_under_5,percent_age_5-9,percent_age_10-14,percent_age_15-19,percent_age_20-24,percent_age_25-34,percent_age_35-44,percent_age_45-54,percent_age_55-59,percent_age_60-64,percent_age_65-74,percent_age_75-84,percent_age85_and_over,percent_age_65_and_over,median_age,population_porportion,percent_age_over_65,percent_age_under_14,percent_low_income
0,60002,19113.0,69.5,5.5,3.2,5.6,78250.0,88649.0,9.4,22.4,24250.0,5.7,7.2,6.4,7.9,4.7,9.9,14.2,17.299999,8.1,6.5,8.2,2.8,1.0,12.0,41.200001,0.001887,12.0,19.299999,8.7


In [9]:
df_out = df[['zip_code', 'percent_employed','percent_no_health',
             'percent_public_health','total_population', 'population_porportion',
             'percent_age_over_65','percent_age_under_14', 'percent_low_income']]


In [10]:
df_out.head(1)

,zip_code,percent_employed,percent_no_health,percent_public_health,total_population,population_porportion,percent_age_over_65,percent_age_under_14,percent_low_income
0,60002,69.5,9.4,22.4,24250.0,0.001887,12.0,19.299999,8.7


In [11]:
df_out.to_csv('../data/demographics.csv')

In [12]:
! ls ../data

WNV_Pop_Age.csv              prediction_probabilities.csv
WNV_Pop_Age_Temp.csv         processed_test.csv
__pycache__                  processed_train.csv
census_age.csv               sampleSubmission.csv
census_info.csv              spray.csv
data-clean.ipynb             spray_traps.csv
demographics.csv             test.csv
distance-mapping.ipynb       train.csv
kaggle_submission.csv        weather.csv
model_data.csv               zip_codes.csv


In [13]:
wnv_age = pd.read_csv('../data/WNV_Pop_Age.csv').rename(columns={'ZipCode': 'zip_code'}).drop('Unnamed: 0', axis=1)


In [14]:
wnv_age.head(1)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,zip_code,poulation,percent_employed,inc_<10k,inc_10-15k,inc_15-25k,median_inc,mean_inc,percent_no_health,percent_public_health,total_population,percent_age_under_5,percent_age_5-9,percent_age_10-14,percent_age_15-19,percent_age_20-24,percent_age_25-34,percent_age_35-44,percent_age_45-54,percent_age_55-59,percent_age_60-64,percent_age_65-74,percent_age_75-84,percent_age85_and_over,percent_age_65_and_over,median_age
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,60634,59366.0,68.0,6.4,4.1,10.1,55767.0,66516.0,21.9,29.9,72718.0,6.1,5.8,5.3,6.0,6.1,16.4,13.1,14.2,7.6,6.6,6.7,4.2,1.9,12.8,38.9


In [15]:
wnv_age.shape

(10506, 38)

In [16]:
df_out = wnv_age.merge(df_out, on=['zip_code'])

In [17]:
df_out.drop(['Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy'])

ValueError: labels ['Address' 'Block' 'Street' 'AddressNumberAndStreet' 'AddressAccuracy'] not contained in axis

In [18]:
! ls ../data/

WNV_Pop_Age.csv              prediction_probabilities.csv
WNV_Pop_Age_Temp.csv         processed_test.csv
WNV_Pop_Age_Temp_Geog.csv    processed_train.csv
__pycache__                  sampleSubmission.csv
census_age.csv               spray.csv
census_info.csv              spray_traps.csv
data-clean.ipynb             test.csv
demographics.csv             train.csv
distance-mapping.ipynb       weather.csv
kaggle_submission.csv        zip_codes.csv
model_data.csv


In [ ]:
wnv_years = []
for i in range(2002,2017):
    wnv_years.append(str(i))
    
wnv_cases = [635, 20, 23, 135, 29, 23, 9, 1, 30, 22, 174, 60, np.NaN, 27, 90]
wnv_deaths = [42, 1, 2, 6, np.NaN, 0, 0, 0, 1, 1, 5, 7, np.NaN, 0, 0]

wnv_casedeaths = pd.DataFrame({"Year": wnv_years, "Cases": wnv_cases, "Deaths" : wnv_deaths})
wnv_casedeaths